# Scanning Session Ingestion

## Setup

### Connect to the database

If you are don't have your login information, contact the administrator.

Using local config file (see [01_pipeline](./01_pipeline.ipynb)):

In [1]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
import datajoint as dj; dj.conn()

from adamacs.pipeline import subject, session, surgery, scan, event, trial, imaging
from adamacs import utility
from adamacs.ingest import behavior as ibe
import numpy as np

[2022-12-15 17:17:23,862][INFO]: Connecting tobiasr@172.26.128.53:3306
[2022-12-15 17:17:23,915][INFO]: Connected tobiasr@172.26.128.53:3306


Loading DLC 2.2.1.1...


/Users/trose/opt/miniconda3/envs/datajoint/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


Manual entry:

In [ ]:
# Manual Entry
import datajoint as dj; import getpass
dj.config['database.host'] = '172.26.128.53'        # Put the server name between these apostrophe
dj.config['database.user'] = 'root'             # Put your user name between these apostrophe
dj.config['database.password'] = getpass.getpass()  # Put your password in the prompt
dj.conn()

from adamacs.pipeline import subject, session, surgery, scan, event, trial, imaging
from adamacs import utility
from adamacs.ingest import behavior as ibe
import numpy as np

## populate scaninfo

In [2]:
scan.ScanInfo.describe()

# general data about the reso/meso scans from header
-> scan.Scan
---
nfields              : tinyint                      # number of fields
nchannels            : tinyint                      # number of channels
ndepths              : int                          # Number of scanning depths (planes)
nframes              : int                          # number of recorded frames
nrois                : tinyint                      # number of ROIs (see scanimage's multi ROI imaging)
x=null               : float                        # (um) ScanImage's 0 point in the motor coordinate system
y=null               : float                        # (um) ScanImage's 0 point in the motor coordinate system
z=null               : float                        # (um) ScanImage's 0 point in the motor coordinate system
fps                  : float                        # (Hz) frames per second - Volumetric Scan Rate
bidirectional        : tinyint                      # true = bidirectional scannin

"# general data about the reso/meso scans from header\n-> scan.Scan\n---\nnfields              : tinyint                      # number of fields\nnchannels            : tinyint                      # number of channels\nndepths              : int                          # Number of scanning depths (planes)\nnframes              : int                          # number of recorded frames\nnrois                : tinyint                      # number of ROIs (see scanimage's multi ROI imaging)\nx=null               : float                        # (um) ScanImage's 0 point in the motor coordinate system\ny=null               : float                        # (um) ScanImage's 0 point in the motor coordinate system\nz=null               : float                        # (um) ScanImage's 0 point in the motor coordinate system\nfps                  : float                        # (Hz) frames per second - Volumetric Scan Rate\nbidirectional        : tinyint                      # true = bidirect

In [3]:
scan.ScanInfo.heading

# general data about the reso/meso scans from header
session_id           : varchar(12)                  # 
scan_id              : varchar(12)                  # 
---
nfields              : tinyint                      # number of fields
nchannels            : tinyint                      # number of channels
ndepths              : int                          # Number of scanning depths (planes)
nframes              : int                          # number of recorded frames
nrois                : tinyint                      # number of ROIs (see scanimage's multi ROI imaging)
x=null               : float                        # (um) ScanImage's 0 point in the motor coordinate system
y=null               : float                        # (um) ScanImage's 0 point in the motor coordinate system
z=null               : float                        # (um) ScanImage's 0 point in the motor coordinate system
fps                  : float                        # (Hz) frames per second - Volume

In [4]:
populate_settings = {'display_progress': True}

In [16]:
scan.ScanInfo.populate(**populate_settings)
scan.ScanInfo()

session_id,scan_id,nfields number of fields,nchannels number of channels,ndepths Number of scanning depths (planes),nframes number of recorded frames,nrois number of ROIs (see scanimage's multi ROI imaging),x (um) ScanImage's 0 point in the motor coordinate system,y (um) ScanImage's 0 point in the motor coordinate system,z (um) ScanImage's 0 point in the motor coordinate system,fps (Hz) frames per second - Volumetric Scan Rate,bidirectional true = bidirectional scanning,usecs_per_line microseconds per scan line,fill_fraction raster scan temporal fill fraction (see scanimage),scan_datetime datetime of the scan,scan_duration (seconds) duration of the scan
sess9FF6TL96,scan9FF6TL96,1,2,1,9250,0,nan,nan,nan,15.3845,1,250.001,0.95,None,601.255


### Creating a Parameter Set

What exactly happens during processing dependso on the parameter set. This is an example of a parameter set and its insert:

In [20]:
# Insert the param_set

# TODO: Parameter set needs to be updated with ScanInfo settings

params_suite2p = {'look_one_level_down': 0.0,
                  'fast_disk': [],
                  'delete_bin': False,
                  'mesoscan': False,
                  'h5py': [],
                  'h5py_key': 'data',
                  'save_path0': [],
                  'subfolders': [],
                  'nplanes': 1,
                  'nchannels': 1,
                  'functional_chan': 1,
                  'tau': 1.0,
                  'fs': 10.0,
                  'force_sktiff': False,
                  'preclassify': 0.0,
                  'save_mat': False,
                  'combined': True,
                  'aspect': 1.0,
                  'do_bidiphase': False,
                  'bidiphase': 0.0,
                  'do_registration': True,
                  'keep_movie_raw': False,
                  'nimg_init': 300,
                  'batch_size': 500,
                  'maxregshift': 0.1,
                  'align_by_chan': 1,
                  'reg_tif': False,
                  'reg_tif_chan2': False,
                  'subpixel': 10,
                  'smooth_sigma': 1.15,
                  'th_badframes': 1.0,
                  'pad_fft': False,
                  'nonrigid': True,
                  'block_size': [128, 128],
                  'snr_thresh': 1.2,
                  'maxregshiftNR': 5.0,
                  '1Preg': False,
                  'spatial_hp': 50.0,
                  'pre_smooth': 2.0,
                  'spatial_taper': 50.0,
                  'roidetect': True,
                  'sparse_mode': False,
                  'diameter': 12,
                  'spatial_scale': 0,
                  'connected': True,
                  'nbinned': 5000,
                  'max_iterations': 20,
                  'threshold_scaling': 1.0,
                  'max_overlap': 0.75,
                  'high_pass': 100.0,
                  'inner_neuropil_radius': 2,
                  'min_neuropil_pixels': 350,
                  'allow_overlap': False,
                  'chan2_thres': 0.65,
                  'baseline': 'maximin',
                  'win_baseline': 60.0,
                  'sig_baseline': 10.0,
                  'prctile_baseline': 8.0,
                  'neucoeff': 0.7,
                  'xrange': np.array([0, 0]),
                  'yrange': np.array([0, 0])}

imaging.ProcessingParamSet.insert_new_params('suite2p', 2, 'basic params', params_suite2p)

DataJointError: The specified param-set already exists - name: 1

In [30]:
imaging.ProcessingTask().delete()

[2022-12-15 17:34:40,522][INFO]: Deleting 1 rows from `tobiasr_imaging`.`processing_task`


Deletes committed.


1

### Create and Run a Processing Task

In [31]:
imaging.ProcessingTask.insert1(('sess9FF6TL96',
                                'scan9FF6TL96',
                                1,
                                'JJ_ROS-1438_2022-11-22_scan9FF6TL96_sess9FF6TL96/suite2p',
                                'trigger'))

To run all unprocessed processing task we call populate on processing:

In [33]:
imaging.Processing.populate(display_progress=True)

Processing:   0%|          | 0/1 [00:00<?, ?it/s]OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


No module named 'cellpose'
cannot use anatomical mode, but otherwise suite2p will run normally
{'data_path': ['/Users/trose/Data/JJ_ROS-1438_2022-11-22_scan9FF6TL96_sess9FF6TL96'], 'tiff_list': ['/Users/trose/Data/JJ_ROS-1438_2022-11-22_scan9FF6TL96_sess9FF6TL96/scan9FF6TL96_JJ_ROS_1438_00002.tif']}
tif
** Found 1 tifs - converting to binary **
2000 frames of binary, time 4.61 sec.
4000 frames of binary, time 5.28 sec.
6000 frames of binary, time 5.93 sec.
8000 frames of binary, time 6.58 sec.
10000 frames of binary, time 7.21 sec.
12000 frames of binary, time 7.83 sec.
14000 frames of binary, time 8.51 sec.
16000 frames of binary, time 9.20 sec.
18000 frames of binary, time 9.85 sec.
time 10.30 sec. Wrote 18500 frames per binary for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not registered / registration forced with ops['do_registration']>1
      (no previous offsets to delete)
----------- REGISTRATION
registering 18500 frames
Reference frame, 11.64 sec.
Regis

Processing: 100%|██████████| 1/1 [07:33<00:00, 453.75s/it]


In [38]:
imaging.scan()

TypeError: 'module' object is not callable

In [39]:
imaging.ProcessingTask()

session_id,scan_id,paramset_idx,processing_output_dir output directory of the processed scan relative to root data directory,"task_mode 'load': load computed analysis results, 'trigger': trigger computation"
sess9FF6TL96,scan9FF6TL96,1,JJ_ROS-1438_2022-11-22_scan9FF6TL96_sess9FF6TL96/suite2p,trigger


In [41]:
imaging.Processing()

session_id,scan_id,paramset_idx,"processing_time time of generation of this set of processed, segmented results",package_version
sess9FF6TL96,scan9FF6TL96,1,2022-12-15 17:42:49,


In [44]:
session_key = (session.Session & 'subject = "ROS-1438"').fetch('KEY')[0]

In [43]:
session.Session()

session_id,subject PyRat import uses this for earmark value,session_datetime
sess9FF6TL96,ROS-1438,2022-11-22 00:00:00


In [45]:
scan.Scan & session_key

session_id,scan_id,scanner,acq_software,scan_notes free-notes
sess9FF6TL96,scan9FF6TL96,mini2p1,ScanImage,


In [46]:
scan.ScanInfo & session_key

session_id,scan_id,nfields number of fields,nchannels number of channels,ndepths Number of scanning depths (planes),nframes number of recorded frames,nrois number of ROIs (see scanimage's multi ROI imaging),x (um) ScanImage's 0 point in the motor coordinate system,y (um) ScanImage's 0 point in the motor coordinate system,z (um) ScanImage's 0 point in the motor coordinate system,fps (Hz) frames per second - Volumetric Scan Rate,bidirectional true = bidirectional scanning,usecs_per_line microseconds per scan line,fill_fraction raster scan temporal fill fraction (see scanimage),scan_datetime datetime of the scan,scan_duration (seconds) duration of the scan
sess9FF6TL96,scan9FF6TL96,1,2,1,9250,0,nan,nan,nan,15.3845,1,250.001,0.95,None,601.255


In [47]:
scan.ScanInfo.Field & session_key

session_id,scan_id,field_idx,px_height height in pixels,px_width width in pixels,um_height height in microns,um_width width in microns,field_x (um) center of field in the motor coordinate system,field_y (um) center of field in the motor coordinate system,field_z (um) relative depth of field,delay_image (ms) delay between the start of the scan and pixels in this field,roi the scanning roi (as recorded in the acquisition software) containing this field - only relevant to mesoscale scans
sess9FF6TL96,scan9FF6TL96,0,256,256,65.7966,65.7966,nan,nan,nan,=BLOB=,None


In [48]:
imaging.ProcessingParamSet()

paramset_idx,processing_method,paramset_desc,param_set_hash,params dictionary of all applicable parameters
1,suite2p,basic params,be32de46-f21d-d714-d131-ec89817de806,=BLOB=


In [49]:
imaging.ProcessingTask * imaging.Processing & session_key

session_id,scan_id,paramset_idx,processing_output_dir output directory of the processed scan relative to root data directory,"task_mode 'load': load computed analysis results, 'trigger': trigger computation","processing_time time of generation of this set of processed, segmented results",package_version
sess9FF6TL96,scan9FF6TL96,1,JJ_ROS-1438_2022-11-22_scan9FF6TL96_sess9FF6TL96/suite2p,trigger,2022-12-15 17:42:49,


In [50]:
imaging.Curation & session_key

session_id,scan_id,paramset_idx,curation_id,curation_time time of generation of this set of curated results,"curation_output_dir output directory of the curated results, relative to root data directory",manual_curation has manual curation been performed on this result?,curation_note
